# LGBM Custom Metrics
This notebook is focused on training our models with the competition metrics.

In [1]:
import numpy as np
import pandas as pd
from utils import read_data, process_time, merge_data, promo_detector, promotionAggregation
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import sys
import xgboost as xgb
import lightgbm as lgb
from datetime import datetime

NUMBER_OF_LAGS = 4

sys.path.append("../../main/datasets/")
!ls  ../../main/datasets/

1.0v  1.0v.zip


## Defining metrics

Evaluation Metric

In [87]:
def feval(prediction, dtrain):
    
    prediction = prediction.astype(int)
    target = dtrain.get_label()
    simulatedPrice = dtrain.get_weight()
#     print("Prediction", prediction.shape)
#     print("Difference", target.shape)
#     print("Maximum", np.minimum( 1 / np.exp(prediction - target),  1))
    return 'feval', -np.sum(prediction  * simulatedPrice * np.minimum( 1 / np.exp(prediction - target),  1)), True

Objective Metric

In [90]:
def gradient(predt, dtrain):
    y = dtrain.get_label()
    sp = dtrain.get_weight()
    print("Gradient", -1 *sp * ((predt <= y) + (predt > y)* (1-predt)/(np.e**(predt - y))))
    return -1 *sp * ((predt <= y) + (predt > y)* (1-predt)/(np.e**(predt - y)))

def hessian(predt, dtrain):
    y = dtrain.get_label()
    sp = dtrain.get_weight() 
    print("Hessian", -1 *sp * ((predt <= y)* 0 +(predt > y)* (2 - predt)* (-1)*(np.e**(sp - predt))))
    return -1 *sp * ((predt <= y)* 0 +(predt > y)* (2 - predt)* (-1)*(np.e**(sp - predt)))

def objective(predt, dtrain):
    grad = gradient(predt, dtrain)
    hess = hessian(predt, dtrain)
    return grad, hess

## Preparing our dataset
These steps were already seen on ```../pre-processing-features``` notebooks.

In [5]:
infos, items, orders = read_data("../../main/datasets/")
print("Sanity checks...", infos.shape, items.shape, orders.shape)

Sanity checks... (10463, 3) (10463, 8) (2181955, 5)


In [6]:
# Changing our time signatures, 
# adding our promotion feature 
# and aggregating our data by weeks...
process_time(orders)
orders = promo_detector(orders)
df = promotionAggregation(orders, items)

In [7]:
def prepareOrders(orders, items):
    """This function is responsible for adding in our 'orders' dataframe
    the items that were not sold. THIS IS NOT MODULARIZED, THUS YOU
    SHOULD CHANGE THE CODE TO BETTER SUIT YOUR DATASET FEATURES
    """
    
    df = orders.copy()
    
    # Getting the IDs that were never sold
    not_sold_items = items[np.logical_not(
        items.itemID.isin(sorted(orders['itemID'].unique())))]

    new_rows = []
    weeks_database = orders['group_backwards'].unique()

    for idd in df['itemID'].unique():
        orders_id = df[df.itemID == idd]
        example = orders_id.iloc[0]

        # finding weeks without itemID sales
        weeks_id = orders_id['group_backwards'].unique()
        weeks_without_id = np.setdiff1d(weeks_database, weeks_id)

        # creating new row
        for w in weeks_without_id:
            new_rows.append({'itemID': idd,
                             'group_backwards': w,
                             'salesPrice_mean': 0,
                             'customerRating': example['customerRating'],
                             'category1': example['category1'],
                             'category2': example['category2'],
                             'category3': example['category3'],
                             'recommendedRetailPrice': example['recommendedRetailPrice'],
                             'orderSum': 0,
                             'manufacturer': example['manufacturer'],
                             'brand': example['brand'],
                             'promotion_mean': 0
                             })
    #  Adding rows in every week with the IDs of the
    # items that were never sold.
    df = df.append(new_rows)
    not_sold_orders = pd.DataFrame()
    for i in range(1, 14):
        aux = not_sold_items.copy()
        aux['group_backwards'] = i
        aux['salesPrice_mean'] = 0
        aux['promotion_mean'] = 0
        aux['orderSum'] = 0
        not_sold_orders = pd.concat([not_sold_orders, aux], axis=0)
    df = pd.concat([df, not_sold_orders], axis=0).sort_values(
        ['group_backwards', 'itemID'], ascending=[False, True], ignore_index=True)
    return df

In [8]:
df = prepareOrders(df, items)

In [9]:
# This cell lags and diffs our features 'orderSum' and "promotion"

shifting = df.copy()

for i in range(1, NUMBER_OF_LAGS + 1):
    # Carrying the data of weeks t-1
    shifting[f'orderSum_{i}'] = shifting.groupby('itemID')['orderSum'].shift(i)
    shifting[f'promotion_mean_{i}'] = shifting.groupby('itemID')['promotion_mean'].shift(i)
    
    # Getting the difference of the orders and promotions between weeks t-1 and t-2...
    shifting[f'orderSum_diff_{i}'] = shifting.groupby('itemID')[f'orderSum_{i}'].diff()
    shifting[f'promotion_mean_diff_{i}'] = shifting.groupby('itemID')[f'promotion_mean_{i}'].diff()
shifting.fillna(0, inplace=True)

## Maximum error
The maximum error we could get in this dataset would be just guessing the mean of our sales from weeks 1 to 12, and that's what the cell below is computing.

In [10]:
worst_possible_prediction = shifting.loc[shifting.group_backwards < 13]['orderSum'].mean()
prediction = np.full(shifting.loc[shifting.group_backwards == 13]['orderSum'].shape, worst_possible_prediction) # Array filled with the mean...
target = shifting.loc[shifting.group_backwards == 13]['orderSum']
print("Guessing the mean of 'orderSum' for all items in target", mse(target, prediction) ** 0.5)

Guessing the mean of 'orderSum' for all items in target 90.29706562119341


## Dataset Splitting
All my experiments will use weeks 13 to 3 as a train set, week 2 as our validation set and week 1 as a test set.

In [11]:
train = shifting.loc[shifting.group_backwards >= 3]
val = shifting.loc[shifting.group_backwards == 2]
test = shifting.loc[shifting.group_backwards == 1]

weights = infos.set_index('itemID')['simulationPrice'].to_dict()

w_train = train['itemID'].map(weights)
w_val = val['itemID'].map(weights)

In [12]:
# I recommend to the other members of the team keeping the
# datatypes of our datasets as Pandas DataFrames instead of Numpy,
# since It will easier to use Boosting Analysis frameworks
y_train = train['orderSum']
y_val = val['orderSum']
X_train = train.drop(columns=["orderSum"])
X_val = val.drop(columns=["orderSum"])

In [88]:
params = {
#           "objective" : "poisson",
#           "metric" :"rmse",
          "learning_rate" : 0.1,
          'verbosity': 2,
          'max_depth': 6,
          'num_leaves': 15,
          "min_data_in_leaf":2000
         }

lgbtrain = lgb.Dataset(X_train, label = y_train, weight=w_train, categorical_feature=[8, 9, 10])
lgbvalid = lgb.Dataset(X_val, label = y_val, weight=w_val, categorical_feature=[8, 9, 10])

num_round = 1000
model = lgb.train(params,
                  lgbtrain,
                  num_round,
                  valid_sets = [lgbtrain, lgbvalid], 
                  verbose_eval=20,
                  early_stopping_rounds=20,
                  fobj=objective,
                  feval=feval)

<ipython-input-86-8b682ce94b1e>:4: RuntimeWarning: invalid value encountered in true_divide
  print("Gradient", -1 *sp * ((predt <= y) + (predt > y)* (1-predt)/(np.e**(predt - y))))
<ipython-input-86-8b682ce94b1e>:5: RuntimeWarning: invalid value encountered in true_divide
  return sp * ((predt <= y) + (predt > y)* (1-predt)/(np.e**(predt - y)))
<ipython-input-86-8b682ce94b1e>:10: RuntimeWarning: overflow encountered in power
  print("Hessian", -1 *sp * ((predt <= y)* 0 +(predt > y)* (2 - predt)* (-1)*(np.e**(sp - predt))))
<ipython-input-86-8b682ce94b1e>:10: RuntimeWarning: invalid value encountered in multiply
  print("Hessian", -1 *sp * ((predt <= y)* 0 +(predt > y)* (2 - predt)* (-1)*(np.e**(sp - predt))))
<ipython-input-86-8b682ce94b1e>:11: RuntimeWarning: overflow encountered in power
  return sp * ((predt <= y)* 0 +(predt > y)* (2 - predt)* (-1)*(np.e**(sp - predt)))
<ipython-input-86-8b682ce94b1e>:11: RuntimeWarning: invalid value encountered in multiply
  return sp * ((predt <

Gradient [  -3.43000007   -9.14999962  -14.03999996 ... -190.52999878 -304.29998779
 -282.16000366]
Hessian [-0. -0. -0. ... -0. -0. -0.]
Training until validation scores don't improve for 20 rounds
Gradient [  -3.43000007   -9.14999962  -14.03999996 ... -190.52999878 -304.29998779
 -282.16000366]
Hessian [-0. -0. -0. ... -0. -0. -0.]
Gradient [  -3.43000007   -9.14999962  -14.03999996 ... -190.52999878 -304.29998779
 -282.16000366]
Hessian [-0. -0. -0. ... -0. -0. -0.]
Gradient [  -3.43000007   -9.14999962  -14.03999996 ... -190.52999878 -304.29998779
 -282.16000366]
Hessian [-0. -0. -0. ... -0. -0. -0.]
Gradient [  -3.43000007   -9.14999962  -14.03999996 ... -190.52999878 -304.29998779
 -282.16000366]
Hessian [-0. -0. -0. ... -0. -0. -0.]
Gradient [  -3.43000007   -9.14999962  -14.03999996 ... -190.52999878 -304.29998779
 -282.16000366]
Hessian [-0. -0. -0. ... -0. -0. -0.]
Gradient [  -3.43000007   -9.14999962  -14.03999996 ... -190.52999878 -304.29998779
 -282.16000366]
Hessian [-0

### Utilities

**Predicting at test time**

In [27]:
y_test = test['orderSum']
X_test = test.drop(columns=["orderSum"])
final_predictions = model.predict(X_test)

In [28]:
final_predictions

array([3.20043400e+01, 2.50993155e-03, 3.38431918e+01, ...,
       2.79345936e-03, 2.79324730e-03, 2.79345936e-03])

In [ ]:
final_predictions[final_predictions < 0].

**Creating our Kaggle CSV**

In [ ]:
final = pd.Series(0, index=np.arange(1, len(items)+1))
final[items.itemID] = final_predictions.astype(int)

final.to_csv("lgbm_kaggle_df.csv", header=["demandPrediction"],
            index_label="itemID", sep="|")

**Saving our model in disk**

In [ ]:
now = datetime.now().strftime("%d-%m-%Y-%Hh%Mm%Ss")
modelName = 'lgbm-' + now
bst.save_model(modelName)